<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#15-14.-프로젝트:-한국어-데이터로-챗봇-만들기" data-toc-modified-id="15-14.-프로젝트:-한국어-데이터로-챗봇-만들기-1">15-14. 프로젝트: 한국어 데이터로 챗봇 만들기</a></span><ul class="toc-item"><li><span><a href="#Step-1.-데이터-수집하기" data-toc-modified-id="Step-1.-데이터-수집하기-1.1">Step 1. 데이터 수집하기</a></span><ul class="toc-item"><li><span><a href="#Loading-data" data-toc-modified-id="Loading-data-1.1.1">Loading data</a></span></li><li><span><a href="#Checking-for-missing-values" data-toc-modified-id="Checking-for-missing-values-1.1.2">Checking for missing values</a></span></li></ul></li><li><span><a href="#Step-2.-데이터-전처리하기" data-toc-modified-id="Step-2.-데이터-전처리하기-1.2">Step 2. 데이터 전처리하기</a></span><ul class="toc-item"><li><span><a href="#Preprocessing-function" data-toc-modified-id="Preprocessing-function-1.2.1">Preprocessing function</a></span></li></ul></li><li><span><a href="#Step-3.-SubwordTextEncoder-사용하기" data-toc-modified-id="Step-3.-SubwordTextEncoder-사용하기-1.3">Step 3. SubwordTextEncoder 사용하기</a></span><ul class="toc-item"><li><span><a href="#SubwordTextEncoder" data-toc-modified-id="SubwordTextEncoder-1.3.1">SubwordTextEncoder</a></span></li><li><span><a href="#START_TOKEN,-END_TOKEN,-VOCAB_SIZE" data-toc-modified-id="START_TOKEN,-END_TOKEN,-VOCAB_SIZE-1.3.2">START_TOKEN, END_TOKEN, VOCAB_SIZE</a></span></li><li><span><a href="#전체-데이터에-대해-정수인코딩과-패딩을-진행" data-toc-modified-id="전체-데이터에-대해-정수인코딩과-패딩을-진행-1.3.3">전체 데이터에 대해 정수인코딩과 패딩을 진행</a></span></li><li><span><a href="#인코더와-디코더의-입력,-레이블-만들기" data-toc-modified-id="인코더와-디코더의-입력,-레이블-만들기-1.3.4">인코더와 디코더의 입력, 레이블 만들기</a></span></li></ul></li><li><span><a href="#Step-4.-모델-구성하기" data-toc-modified-id="Step-4.-모델-구성하기-1.4">Step 4. 모델 구성하기</a></span><ul class="toc-item"><li><span><a href="#PositionalEncoding()" data-toc-modified-id="PositionalEncoding()-1.4.1">PositionalEncoding()</a></span></li><li><span><a href="#scaled_dot_product_attention()" data-toc-modified-id="scaled_dot_product_attention()-1.4.2">scaled_dot_product_attention()</a></span></li><li><span><a href="#MultiHeadAttention()" data-toc-modified-id="MultiHeadAttention()-1.4.3">MultiHeadAttention()</a></span></li><li><span><a href="#create_padding_make()" data-toc-modified-id="create_padding_make()-1.4.4">create_padding_make()</a></span></li><li><span><a href="#create_look_ahead_mask()" data-toc-modified-id="create_look_ahead_mask()-1.4.5">create_look_ahead_mask()</a></span></li><li><span><a href="#encoder_layer()" data-toc-modified-id="encoder_layer()-1.4.6">encoder_layer()</a></span></li><li><span><a href="#encoder()" data-toc-modified-id="encoder()-1.4.7">encoder()</a></span></li><li><span><a href="#decoder_layer()" data-toc-modified-id="decoder_layer()-1.4.8">decoder_layer()</a></span></li><li><span><a href="#decoder()" data-toc-modified-id="decoder()-1.4.9">decoder()</a></span></li><li><span><a href="#trainsformer()" data-toc-modified-id="trainsformer()-1.4.10">trainsformer()</a></span></li><li><span><a href="#Hyper-parameter" data-toc-modified-id="Hyper-parameter-1.4.11">Hyper parameter</a></span></li><li><span><a href="#Loss-function" data-toc-modified-id="Loss-function-1.4.12">Loss function</a></span></li><li><span><a href="#Custom-Learning-rate-Scheduling" data-toc-modified-id="Custom-Learning-rate-Scheduling-1.4.13">Custom Learning rate Scheduling</a></span></li><li><span><a href="#Compile" data-toc-modified-id="Compile-1.4.14">Compile</a></span></li><li><span><a href="#Training" data-toc-modified-id="Training-1.4.15">Training</a></span></li></ul></li><li><span><a href="#Step-5.-모델-평가하기" data-toc-modified-id="Step-5.-모델-평가하기-1.5">Step 5. 모델 평가하기</a></span></li><li><span><a href="#Conclusion" data-toc-modified-id="Conclusion-1.6">Conclusion</a></span></li><li><span><a href="#Postscript" data-toc-modified-id="Postscript-1.7">Postscript</a></span></li><li><span><a href="#Reference" data-toc-modified-id="Reference-1.8">Reference</a></span></li></ul></li></ul></div>

# 15-14. 프로젝트: 한국어 데이터로 챗봇 만들기

영어로 만들었던 챗봇을 한국어 데이터로 바꿔서 훈련시켜봅시다.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import tensorflow_datasets as tfds
import os
import re

In [2]:
strategy = tf.distribute.MirroredStrategy()

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0', '/job:localhost/replica:0/task:0/device:GPU:1')


## Step 1. 데이터 수집하기
---
한국어 챗봇 데이터는 송영숙님이 공개한 챗봇 데이터를 사용합니다.

이 데이터는 아래의 링크에서 다운로드할 수 있습니다.

- [songys/Chatbot_data](https://github.com/songys/Chatbot_data/blob/master/ChatbotData%20.csv)

```
wget으로 데이터 다운로드
$ wget https://github.com/songys/Chatbot_data/raw/master/ChatbotData%20.csv
```

### Loading data

In [3]:
filepath = os.path.join(os.getenv('HOME'), 'aiffel/songys_chatbot/ChatbotData .csv')
train_data = pd.read_csv(filepath)
train_data

,Q,A,label
0,12시 땡!,하루가 또 가네요.,0
1,1지망 학교 떨어졌어,위로해 드립니다.,0
2,3박4일 놀러가고 싶다,여행은 언제나 좋죠.,0
3,3박4일 정도 놀러가고 싶다,여행은 언제나 좋죠.,0
4,PPL 심하네,눈살이 찌푸려지죠.,0
...,...,...,...
11818,훔쳐보는 것도 눈치 보임.,티가 나니까 눈치가 보이는 거죠!,2
11819,훔쳐보는 것도 눈치 보임.,훔쳐보는 거 티나나봐요.,2
11820,흑기사 해주는 짝남.,설렜겠어요.,2
11821,힘든 연애 좋은 연애라는게 무슨 차이일까?,잘 헤어질 수 있는 사이 여부인 거 같아요.,2


### Checking for missing values

In [4]:
train_data.isnull().sum()

Q        0
A        0
label    0
dtype: int64

## Step 2. 데이터 전처리하기
---
영어 데이터와는 전혀 다른 데이터인 만큼 영어 데이터에 사용했던 전처리와 일부 동일한 전처리도 필요하겠지만 전체적으로는 다른 전처리를 수행해야 할 수도 있습니다.

### Preprocessing function

In [5]:
def preprocess_sentence(sentence):
    """
    Preprocessing function.
    Make the distance between words and punctuation marks.
    """
    sentence = re.sub(r"([?.!,])", r" \1 ", sentence)
    sentence = sentence.strip()
    return sentence

In [6]:
questions = [preprocess_sentence(sentence) for sentence in train_data['Q']]
answers = [preprocess_sentence(sentence) for sentence in train_data['A']]

In [7]:
print(questions[:5])
print(answers[:5])

['12시 땡 !', '1지망 학교 떨어졌어', '3박4일 놀러가고 싶다', '3박4일 정도 놀러가고 싶다', 'PPL 심하네']
['하루가 또 가네요 .', '위로해 드립니다 .', '여행은 언제나 좋죠 .', '여행은 언제나 좋죠 .', '눈살이 찌푸려지죠 .']


## Step 3. SubwordTextEncoder 사용하기
---
한국어 데이터는 형태소 분석기를 사용하여 토크나이징을 해야 한다고 많은 분이 알고 있습니다. 하지만 여기서는 형태소 분석기가 아닌 위 실습에서 사용했던 내부 단어 토크나이저인 `SubwordTextEncoder`를 그대로 사용해보세요.

### SubwordTextEncoder

In [8]:
# SubwordTextEncoder를 사용하여 질문, 답변 데이터로부터 단어집합 생성
tokenizer = tfds.deprecated.text.SubwordTextEncoder.build_from_corpus(
    questions + answers, target_vocab_size=2**13)

### START_TOKEN, END_TOKEN, VOCAB_SIZE

In [9]:
START_TOKEN, END_TOKEN = [tokenizer.vocab_size], [tokenizer.vocab_size+1]
VOCAB_SIZE = tokenizer.vocab_size+2

In [10]:
print(f'START_TOKEN: {START_TOKEN}')
print(f'END_TOKEN: {END_TOKEN}')
print(f'VOCAB_SIZE: {VOCAB_SIZE}')

START_TOKEN: [8178]
END_TOKEN: [8179]
VOCAB_SIZE: 8180


### 전체 데이터에 대해 정수인코딩과 패딩을 진행

In [11]:
# 샘플의 최대 허용 길이 또는 패딩 후의 최종 길이
MAX_LENGTH = 40

In [12]:
def tokenize_and_filter(inputs, outputs):
    """
    Integer encoding,
    removing samples exceeding maximum length,
    padding
    """
    tokenized_inputs, tokenized_outputs = [], []

    for (sentence1, sentence2) in zip(inputs, outputs):
        # 정수 인코딩 과정에서 시작 토큰과 종료 토큰을 추가
        sentence1 = START_TOKEN + tokenizer.encode(sentence1) + END_TOKEN
        sentence2 = START_TOKEN + tokenizer.encode(sentence2) + END_TOKEN

        # 최대 길이 40 이하인 경우에만 데이터셋으로 허용
        if len(sentence1) <= MAX_LENGTH and len(sentence2) <= MAX_LENGTH:
            tokenized_inputs.append(sentence1)
            tokenized_outputs.append(sentence2)

    # 최대 길이 40으로 모든 데이터셋을 패딩
    tokenized_inputs = tf.keras.preprocessing.sequence.pad_sequences(
        tokenized_inputs, maxlen=MAX_LENGTH, padding='post')
    tokenized_outputs = tf.keras.preprocessing.sequence.pad_sequences(
        tokenized_outputs, maxlen=MAX_LENGTH, padding='post')

    return tokenized_inputs, tokenized_outputs

In [13]:
questions, answers = tokenize_and_filter(questions, answers)

In [14]:
print(f'questions.shape: {questions.shape}')
print(f'answers.shape: {answers.shape}')

questions.shape: (11823, 40)
answers.shape: (11823, 40)


### 인코더와 디코더의 입력, 레이블 만들기

In [15]:
BATCH_SIZE = 64
BUFFER_SIZE = 20000

dataset = tf.data.Dataset.from_tensor_slices((
    {'inputs': questions,
     'dec_inputs': answers[:, :-1]},  # 디코더의 입력
    {'outputs': answers[:, 1:]},      # 시작 토큰 제거
))

dataset = dataset.cache()
dataset = dataset.shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE)
dataset = dataset.prefetch(tf.data.experimental.AUTOTUNE)

## Step 4. 모델 구성하기
---
위 실습 내용을 참고하여 트랜스포머 모델을 구현합니다.

### PositionalEncoding()

In [16]:
class PositionalEncoding(tf.keras.layers.Layer):

    def __init__(self, position, d_model):
        super(PositionalEncoding, self).__init__()
        self.pos_encoding = self.positional_encoding(position, d_model)

    def get_angles(self, position, i, d_model):
        angles = 1 / tf.pow(10000, (2 * (i // 2)) /
                            tf.cast(d_model, tf.float32))
        return position * angles

    def positional_encoding(self, position, d_model):
        angle_rads = self.get_angles(
            position=tf.range(position, dtype=tf.float32)[:, tf.newaxis],
            i=tf.range(d_model, dtype=tf.float32)[tf.newaxis, :],
            d_model=d_model)

        # 배열의 짝수 인덱스(2i)에는 sin 함수 적용
        sines = tf.math.sin(angle_rads[:, 0::2])

        # 배열의 홀수 인덱스(2i+1)에는 cosine 함수 적용
        cosines = tf.math.cos(angle_rads[:, 1::2])

        pos_encoding = tf.concat([sines, cosines], axis=-1)
        pos_encoding = pos_encoding[tf.newaxis, ...]
        return tf.cast(pos_encoding, tf.float32)

    def call(self, inputs):
        return inputs + self.pos_encoding[:, :tf.shape(inputs)[1], :]

### scaled_dot_product_attention()

In [17]:
def scaled_dot_product_attention(query, key, value, mask):
    """Calculate the attention weight."""
    matmul_qk = tf.matmul(query, key, transpose_b=True)

    # scale matmul_qk
    depth = tf.cast(tf.shape(key)[-1], tf.float32)
    logits = matmul_qk / tf.math.sqrt(depth)

    # add the mask to zero out padding tokens
    if mask is not None:
        logits += (mask * -1e9)

    # softmax is normalized on the last axis (seq_len_k)
    attention_weights = tf.nn.softmax(logits, axis=-1)

    output = tf.matmul(attention_weights, value)

    return output

### MultiHeadAttention()

In [18]:
class MultiHeadAttention(tf.keras.layers.Layer):

    def __init__(self, d_model, num_heads, name="multi_head_attention"):
        super(MultiHeadAttention, self).__init__(name=name)
        self.num_heads = num_heads
        self.d_model = d_model

        assert d_model % self.num_heads == 0

        self.depth = d_model // self.num_heads

        self.query_dense = tf.keras.layers.Dense(units=d_model)
        self.key_dense = tf.keras.layers.Dense(units=d_model)
        self.value_dense = tf.keras.layers.Dense(units=d_model)

        self.dense = tf.keras.layers.Dense(units=d_model)

    def split_heads(self, inputs, batch_size):
        inputs = tf.reshape(
            inputs, shape=(batch_size, -1, self.num_heads, self.depth))
        return tf.transpose(inputs, perm=[0, 2, 1, 3])

    def call(self, inputs):
        query, key, value, mask = inputs['query'], inputs['key'], inputs[
            'value'], inputs['mask']
        batch_size = tf.shape(query)[0]

        # linear layers
        query = self.query_dense(query)
        key = self.key_dense(key)
        value = self.value_dense(value)

        # 병렬 연산을 위한 머리를 여러 개 만듭니다.
        query = self.split_heads(query, batch_size)
        key = self.split_heads(key, batch_size)
        value = self.split_heads(value, batch_size)

        # 스케일드 닷 프로덕트 어텐션 함수
        scaled_attention = scaled_dot_product_attention(
            query, key, value, mask)

        scaled_attention = tf.transpose(scaled_attention, perm=[0, 2, 1, 3])

        # 어텐션 연산 후에 각 결과를 다시 연결(concatenate)합니다.
        concat_attention = tf.reshape(scaled_attention,
                                      (batch_size, -1, self.d_model))

        # final linear layer
        outputs = self.dense(concat_attention)

        return outputs

### create_padding_make()

In [19]:
def create_padding_mask(x):
    mask = tf.cast(tf.math.equal(x, 0), tf.float32)
    # (batch_size, 1, 1, sequence length)
    return mask[:, tf.newaxis, tf.newaxis, :]

### create_look_ahead_mask()

In [20]:
def create_look_ahead_mask(x):
    seq_len = tf.shape(x)[1]
    look_ahead_mask = 1 - \
        tf.linalg.band_part(tf.ones((seq_len, seq_len)), -1, 0)
    padding_mask = create_padding_mask(x)
    return tf.maximum(look_ahead_mask, padding_mask)

### encoder_layer()

In [21]:
# 인코더 하나의 레이어를 함수로 구현.
# 이 하나의 레이어 안에는 두 개의 서브 레이어가 존재합니다.
def encoder_layer(units, d_model, num_heads, dropout, name="encoder_layer"):
    inputs = tf.keras.Input(shape=(None, d_model), name="inputs")

    # 패딩 마스크 사용
    padding_mask = tf.keras.Input(shape=(1, 1, None), name="padding_mask")

    # 첫 번째 서브 레이어 : 멀티 헤드 어텐션 수행 (셀프 어텐션)
    attention = MultiHeadAttention(
        d_model, num_heads, name="attention")({
            'query': inputs,
            'key': inputs,
            'value': inputs,
            'mask': padding_mask
        })

    # 어텐션의 결과는 Dropout과 Layer Normalization이라는 훈련을 돕는 테크닉을 수행
    attention = tf.keras.layers.Dropout(rate=dropout)(attention)
    attention = tf.keras.layers.LayerNormalization(
        epsilon=1e-6)(inputs + attention)

    # 두 번째 서브 레이어 : 2개의 완전연결층
    outputs = tf.keras.layers.Dense(units=units, activation='relu')(attention)
    outputs = tf.keras.layers.Dense(units=d_model)(outputs)

    # 완전연결층의 결과는 Dropout과 LayerNormalization이라는 훈련을 돕는 테크닉을 수행
    outputs = tf.keras.layers.Dropout(rate=dropout)(outputs)
    outputs = tf.keras.layers.LayerNormalization(
        epsilon=1e-6)(attention + outputs)

    return tf.keras.Model(
        inputs=[inputs, padding_mask], outputs=outputs, name=name)

### encoder()

In [22]:
def encoder(vocab_size,
            num_layers,
            units,
            d_model,
            num_heads,
            dropout,
            name="encoder"):
    inputs = tf.keras.Input(shape=(None,), name="inputs")

    # 패딩 마스크 사용
    padding_mask = tf.keras.Input(shape=(1, 1, None), name="padding_mask")

    # 임베딩 레이어
    embeddings = tf.keras.layers.Embedding(vocab_size, d_model)(inputs)
    embeddings *= tf.math.sqrt(tf.cast(d_model, tf.float32))

    # 포지셔널 인코딩
    embeddings = PositionalEncoding(vocab_size, d_model)(embeddings)

    outputs = tf.keras.layers.Dropout(rate=dropout)(embeddings)

    # num_layers만큼 쌓아올린 인코더의 층.
    for i in range(num_layers):
        outputs = encoder_layer(
            units=units,
            d_model=d_model,
            num_heads=num_heads,
            dropout=dropout,
            name="encoder_layer_{}".format(i),
        )([outputs, padding_mask])

    return tf.keras.Model(
        inputs=[inputs, padding_mask], outputs=outputs, name=name)

### decoder_layer()

In [23]:
# 디코더 하나의 레이어를 함수로 구현.
# 이 하나의 레이어 안에는 세 개의 서브 레이어가 존재합니다.
def decoder_layer(units, d_model, num_heads, dropout, name="decoder_layer"):
    inputs = tf.keras.Input(shape=(None, d_model), name="inputs")
    enc_outputs = tf.keras.Input(shape=(None, d_model), name="encoder_outputs")
    look_ahead_mask = tf.keras.Input(
        shape=(1, None, None), name="look_ahead_mask")
    padding_mask = tf.keras.Input(shape=(1, 1, None), name='padding_mask')

    # 첫 번째 서브 레이어 : 멀티 헤드 어텐션 수행 (셀프 어텐션)
    attention1 = MultiHeadAttention(
        d_model, num_heads, name="attention_1")(inputs={
            'query': inputs,
            'key': inputs,
            'value': inputs,
            'mask': look_ahead_mask
        })

    # 멀티 헤드 어텐션의 결과는 LayerNormalization이라는 훈련을 돕는 테크닉을 수행
    attention1 = tf.keras.layers.LayerNormalization(
        epsilon=1e-6)(attention1 + inputs)

    # 두 번째 서브 레이어 : 마스크드 멀티 헤드 어텐션 수행 (인코더-디코더 어텐션)
    attention2 = MultiHeadAttention(
        d_model, num_heads, name="attention_2")(inputs={
            'query': attention1,
            'key': enc_outputs,
            'value': enc_outputs,
            'mask': padding_mask
        })

    # 마스크드 멀티 헤드 어텐션의 결과는
    # Dropout과 LayerNormalization이라는 훈련을 돕는 테크닉을 수행
    attention2 = tf.keras.layers.Dropout(rate=dropout)(attention2)
    attention2 = tf.keras.layers.LayerNormalization(
        epsilon=1e-6)(attention2 + attention1)

    # 세 번째 서브 레이어 : 2개의 완전연결층
    outputs = tf.keras.layers.Dense(units=units, activation='relu')(attention2)
    outputs = tf.keras.layers.Dense(units=d_model)(outputs)

    # 완전연결층의 결과는 Dropout과 LayerNormalization 수행
    outputs = tf.keras.layers.Dropout(rate=dropout)(outputs)
    outputs = tf.keras.layers.LayerNormalization(
        epsilon=1e-6)(outputs + attention2)

    return tf.keras.Model(
        inputs=[inputs, enc_outputs, look_ahead_mask, padding_mask],
        outputs=outputs,
        name=name)

### decoder()

In [24]:
def decoder(vocab_size,
            num_layers,
            units,
            d_model,
            num_heads,
            dropout,
            name='decoder'):
    inputs = tf.keras.Input(shape=(None,), name='inputs')
    enc_outputs = tf.keras.Input(shape=(None, d_model), name='encoder_outputs')
    look_ahead_mask = tf.keras.Input(
        shape=(1, None, None), name='look_ahead_mask')

    # 패딩 마스크
    padding_mask = tf.keras.Input(shape=(1, 1, None), name='padding_mask')

    # 임베딩 레이어
    embeddings = tf.keras.layers.Embedding(vocab_size, d_model)(inputs)
    embeddings *= tf.math.sqrt(tf.cast(d_model, tf.float32))

    # 포지셔널 인코딩
    embeddings = PositionalEncoding(vocab_size, d_model)(embeddings)

    # Dropout이라는 훈련을 돕는 테크닉을 수행
    outputs = tf.keras.layers.Dropout(rate=dropout)(embeddings)

    for i in range(num_layers):
        outputs = decoder_layer(
            units=units,
            d_model=d_model,
            num_heads=num_heads,
            dropout=dropout,
            name='decoder_layer_{}'.format(i),
        )(inputs=[outputs, enc_outputs, look_ahead_mask, padding_mask])

    return tf.keras.Model(
        inputs=[inputs, enc_outputs, look_ahead_mask, padding_mask],
        outputs=outputs,
        name=name)

### trainsformer()

In [25]:
def transformer(vocab_size,
                num_layers,
                units,
                d_model,
                num_heads,
                dropout,
                name="transformer"):
    """
    TRANSFORMER
    """
    inputs = tf.keras.Input(shape=(None,), name="inputs")
    dec_inputs = tf.keras.Input(shape=(None,), name="dec_inputs")

    # 인코더에서 패딩을 위한 마스크
    enc_padding_mask = tf.keras.layers.Lambda(
        create_padding_mask,
        output_shape=(1, 1, None),
        name='enc_padding_mask')(inputs)

    # 디코더에서 미래의 토큰을 마스크 하기 위해서 사용합니다.
    # 내부적으로 패딩 마스크도 포함되어져 있습니다.
    look_ahead_mask = tf.keras.layers.Lambda(
        create_look_ahead_mask,
        output_shape=(1, None, None),
        name='look_ahead_mask')(dec_inputs)

    # 두 번째 어텐션 블록에서 인코더의 벡터들을 마스킹
    # 디코더에서 패딩을 위한 마스크
    dec_padding_mask = tf.keras.layers.Lambda(
        create_padding_mask,
        output_shape=(1, 1, None),
        name='dec_padding_mask')(inputs)

    # 인코더
    enc_outputs = encoder(
        vocab_size=vocab_size,
        num_layers=num_layers,
        units=units,
        d_model=d_model,
        num_heads=num_heads,
        dropout=dropout,
    )(inputs=[inputs, enc_padding_mask])

    # 디코더
    dec_outputs = decoder(
        vocab_size=vocab_size,
        num_layers=num_layers,
        units=units,
        d_model=d_model,
        num_heads=num_heads,
        dropout=dropout,
    )(inputs=[dec_inputs, enc_outputs, look_ahead_mask, dec_padding_mask])

    # 완전연결층
    outputs = tf.keras.layers.Dense(
        units=vocab_size, name="outputs")(dec_outputs)

    return tf.keras.Model(inputs=[inputs, dec_inputs], outputs=outputs, name=name)

### Hyper parameter

In [26]:
# 하이퍼파라미터
tf.keras.backend.clear_session()

# 하이퍼파라미터
NUM_LAYERS = 6  # 인코더와 디코더의 층의 개수
D_MODEL = 512  # 인코더와 디코더 내부의 입, 출력의 고정 차원
NUM_HEADS = 8  # 멀티 헤드 어텐션에서의 헤드 수
UNITS = 2048  # 피드 포워드 신경망의 은닉층의 크기
DROPOUT = 0.1  # 드롭아웃의 비율

나는 논문과 똑같이 해보겠다는 패기

In [27]:
with strategy.scope():
    model = transformer(
        vocab_size=VOCAB_SIZE,
        num_layers=NUM_LAYERS,
        units=UNITS,
        d_model=D_MODEL,
        num_heads=NUM_HEADS,
        dropout=DROPOUT)

model.summary()

Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Index'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Index'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Index'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Index'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Index'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Index'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Model: "transformer"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
inputs (InputLayer)             [(None, None)]       0                                            
__________________________________________________________________________________________________
dec_inputs (InputLayer)         [(None, None)]       0                                            
__________________________________________________________________________________________________
enc_padding_mask (Lambda)       (None, 1, 1, None)   0           inputs[0][0]        

### Loss function

In [28]:
def loss_function(y_true, y_pred):
    y_true = tf.reshape(y_true, shape=(-1, MAX_LENGTH - 1))

    loss = tf.keras.losses.SparseCategoricalCrossentropy(
        from_logits=True, reduction='none')(y_true, y_pred)

    mask = tf.cast(tf.not_equal(y_true, 0), tf.float32)
    loss = tf.multiply(loss, mask)

    return tf.reduce_mean(loss)

### Custom Learning rate Scheduling

$lrate = d_{model}^{-0.5} \cdot min(step\_num^{-0.5}, step\_num \cdot warmup\_steps^{-1.5})$

In [29]:
class CustomSchedule(tf.keras.optimizers.schedules.LearningRateSchedule):

    def __init__(self, d_model, warmup_steps=4000):
        super(CustomSchedule, self).__init__()

        self.d_model = d_model
        self.d_model = tf.cast(self.d_model, tf.float32)

        self.warmup_steps = warmup_steps

    def __call__(self, step):
        arg1 = tf.math.rsqrt(step)
        arg2 = step * (self.warmup_steps**-1.5)

        return tf.math.rsqrt(self.d_model) * tf.math.minimum(arg1, arg2)

### Compile

In [30]:
learning_rate = CustomSchedule(D_MODEL)

optimizer = tf.keras.optimizers.Adam(
    learning_rate, beta_1=0.9, beta_2=0.98, epsilon=1e-9)


def accuracy(y_true, y_pred):
    y_true = tf.reshape(y_true, shape=(-1, MAX_LENGTH - 1))
    return tf.keras.metrics.sparse_categorical_accuracy(y_true, y_pred)

with strategy.scope():
    model.compile(optimizer=optimizer, loss=loss_function, metrics=[accuracy])

INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).


INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).


INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).


INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).


INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).


INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).


INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).


INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).


### Training

In [31]:
EPOCHS = 50
model.fit(dataset, epochs=EPOCHS, verbose=1)

Epoch 1/50
INFO:tensorflow:batch_all_reduce: 254 all-reduces with algorithm = nccl, num_packs = 1


INFO:tensorflow:batch_all_reduce: 254 all-reduces with algorithm = nccl, num_packs = 1


INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:GPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:GPU:0', '/job:localhost/replica:0/task:0/device:GPU:1').


INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:GPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:GPU:0', '/job:localhost/replica:0/task:0/device:GPU:1').


INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:GPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:GPU:0', '/job:localhost/replica:0/task:0/device:GPU:1').


...

INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).


INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).


INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).


INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).


185/185 [==============================] - 56s 118ms/step - loss: 1.4477 - accuracy: 0.0180
Epoch 2/50
185/185 [==============================] - 22s 121ms/step - loss: 1.1016 - accuracy: 0.0496
Epoch 3/50
185/185 [==============================] - 22s 121ms/step - loss: 0.9846 - accuracy: 0.0508
Epoch 4/50
185/185 [==============================] - 23s 122ms/step - loss: 0.9451 - accuracy: 0.0530
Epoch 5/50
185/185 [==============================] - 23s 126ms/step - loss: 0.8975 - accuracy: 0.0555
Epoch 6/50
185/185 [==============================] - 24s 128ms/step - loss: 0.8515 - accuracy: 0.0580
Epoch 7/50
185/185 [==============================] - 24s 129ms/step - loss: 0.8089 - accuracy: 0.0606
Epoch 8/50
185/185 [==============================] - 24s 130ms/step - loss: 0.7556 - accuracy: 0.0643
Epoch 9/50
185/185 [==============================] - 24s 131ms/step - loss: 0.7039 - accuracy: 0.0689
Epoch 10/50
185/185 [==============================] - 24s 131ms/step - loss: 0.6414

## Step 5. 모델 평가하기
---
Step 1에서 선택한 전처리 방법을 고려하여 입력된 문장에 대해서 대답을 얻는 예측 함수를 만듭니다.

In [32]:
def decoder_inference(sentence):
    sentence = preprocess_sentence(sentence)

    # 입력된 문장을 정수 인코딩 후, 시작 토큰과 종료 토큰을 앞뒤로 추가.
    # ex) Where have you been? → [[8331   86   30    5 1059    7 8332]]
    sentence = tf.expand_dims(
        START_TOKEN + tokenizer.encode(sentence) + END_TOKEN, axis=0)

    # 디코더의 현재까지의 예측한 출력 시퀀스가 지속적으로 저장되는 변수.
    # 처음에는 예측한 내용이 없음으로 시작 토큰만 별도 저장. ex) 8331
    output_sequence = tf.expand_dims(START_TOKEN, 0)

    # 디코더의 인퍼런스 단계
    for i in range(MAX_LENGTH):
        # 디코더는 최대 MAX_LENGTH의 길이만큼 다음 단어 예측을 반복합니다.
        predictions = model(inputs=[sentence, output_sequence], training=False)
        predictions = predictions[:, -1:, :]

        # 현재 예측한 단어의 정수
        predicted_id = tf.cast(tf.argmax(predictions, axis=-1), tf.int32)

        # 만약 현재 예측한 단어가 종료 토큰이라면 for문을 종료
        if tf.equal(predicted_id, END_TOKEN[0]):
            break

        # 예측한 단어들은 지속적으로 output_sequence에 추가됩니다.
        # 이 output_sequence는 다시 디코더의 입력이 됩니다.
        output_sequence = tf.concat([output_sequence, predicted_id], axis=-1)

    return tf.squeeze(output_sequence, axis=0)

In [33]:
def sentence_generation(sentence):
    # 입력 문장에 대해서 디코더를 동작 시켜 예측된 정수 시퀀스를 리턴받습니다.
    prediction = decoder_inference(sentence)

    # 정수 시퀀스를 다시 텍스트 시퀀스로 변환합니다.
    predicted_sentence = tokenizer.decode(
        [i for i in prediction if i < tokenizer.vocab_size])

    print('입력 : {}'.format(sentence))
    print('출력 : {}'.format(predicted_sentence))

    return predicted_sentence

In [34]:
sentence_generation('누구냐 넌?!')

입력 : 누구냐 넌?!
출력 : 그럴 수 있을 거예요 .


'그럴 수 있을 거예요 .'

In [35]:
sentence_generation('커피는 역시 모닝커피지.')

입력 : 커피는 역시 모닝커피지.
출력 : 좋은 시간 보내고 있나봐요 .


'좋은 시간 보내고 있나봐요 .'

In [36]:
sentence_generation('여행가고 싶다.')

입력 : 여행가고 싶다.
출력 : 오늘 피크닉 가기 좋은 날이에요 .


'오늘 피크닉 가기 좋은 날이에요 .'

In [37]:
sentence_generation('내일 날씨는 어떨 것 같아?')

입력 : 내일 날씨는 어떨 것 같아?
출력 : 저는 둘이 가는 게 좋아요 .


'저는 둘이 가는 게 좋아요 .'

In [38]:
sentence_generation('야식으로 뭐 먹을까?')

입력 : 야식으로 뭐 먹을까?
출력 : 당신이라면 충분히 극복할 수 있어요 .


'당신이라면 충분히 극복할 수 있어요 .'

In [39]:
sentence_generation('좋아하는 색은?')

입력 : 좋아하는 색은?
출력 : 내가 결혼할 때가 결혼적령기예요 .


'내가 결혼할 때가 결혼적령기예요 .'

In [40]:
sentence_generation('주말에 같이 드라이브 갈래?')

입력 : 주말에 같이 드라이브 갈래?
출력 : 혼자 사는 것보다 불편하겠죠 .


'혼자 사는 것보다 불편하겠죠 .'

In [41]:
sentence_generation('너 자꾸 헛소리 할래?')

입력 : 너 자꾸 헛소리 할래?
출력 : 내가 우선순위면 돼요 .


'내가 우선순위면 돼요 .'

In [42]:
sentence_generation('이기적이구나?')

입력 : 이기적이구나?
출력 : 그게 진짜 사랑이네요 .


'그게 진짜 사랑이네요 .'

In [43]:
sentence_generation('네가 사랑을 알아?')

입력 : 네가 사랑을 알아?
출력 : 제가 있잖아요 .


'제가 있잖아요 .'

In [44]:
sentence_generation('그만하자. 오늘은 여기까지.')

입력 : 그만하자. 오늘은 여기까지.
출력 : 더 좋은 곳에서 살 수 있을 거예요 .


'더 좋은 곳에서 살 수 있을 거예요 .'

In [45]:
sentence_generation('싸우자!')

입력 : 싸우자!
출력 : 같이 가는 여행도 좋지만 혼자 여행도 좋을 거예요 .


'같이 가는 여행도 좋지만 혼자 여행도 좋을 거예요 .'

## Conclusion
---
- 한국어 전처리를 통해 학습 데이터셋을 구축하였다.  
    - 공백과 특수문자 처리, 토크나이징, 병렬데이터 구축의 과정이 적절히 진행되었다.

- 트랜스포머 모델을 구현하여 한국어 챗봇 모델 학습을 정상적으로 진행하였다.  
    - 구현한 트랜스포머 모델이 한국어 병렬 데이터 학습 시 안정적으로 수렴하였다.

- 한국어 입력문장에 대해 한국어로 답변하는 함수를 구현하였다.  
    - 한국어 입력문장에 그럴듯한 한국어로 답변을 리턴하였다.

## Postscript
---

- 논문과 같은 하이퍼 파라미터를 적용하면 더 똑똑하게 답변할 줄 알았는데, 생각보다 엉뚱한 소리를 많이 합니다. 역시 중요한 것은 학습 데이터인가 봅니다.
- 이미 공개된 코드가 있어 구현에는 전혀 어려움이 없었습니다.
- 새로운 모델에 대한 공부를 할 수 있는 좋은 시간이었습니다.
- 이번 실습을 통해 자연어처리에 대한 흥미가 생겼습니다.

## Reference
---
- [2) 트랜스포머를 이용한 한국어 챗봇(Transformer Chatbot Tutorial) - 딥 러닝을 이용한 자연어 처리 입문](https://wikidocs.net/89786)